## Train an classifier for hyperspectral image


This example is developed in Google Colab. Connect Google Cloud Drive to your Colab to load the dataset later.

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['Colab Notebooks',
 'Indian_pines_gt.mat',
 'Indian_pines_corrected.mat',
 '1204',
 'data']

Install the enviroment of ColossalAI.

In [2]:
!pip install ColossalAI deepspeed

     |████████████████████████████████| 234 kB 5.3 MB/s 
     |████████████████████████████████| 517 kB 40.1 MB/s 
     |████████████████████████████████| 124 kB 47.4 MB/s 
     |████████████████████████████████| 108 kB 42.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 18.2 MB 235 kB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.8-py3-none-any.whl size=532154 sha256=61415a9c5a0f031ab305c7b0f80be29f217cce3dfc8647619621cd98cca5aff8
  Stored in directory: /root/.cache/pip/wheels/8d/c6/82/cabd9a300c582a221591fd2c8c997e1f03f601e748aad44e4e
Successfully built deepspeed


In [3]:
import colossalai
from colossalai.engine import Engine, NoPipelineSchedule
from colossalai.trainer import Trainer
from colossalai.context import Config
import torch

Colossalai should be built with cuda extension to use the FP16 optimizer
Colossalai should be built with cuda extension to use the FP16 optimizer
apex is required for mixed precision training


Initialize distributed environment for compatibility (we just set the number of parallel processes to 1 for single GPU.)

In [4]:
parallel_cfg = Config(dict(parallel=dict(
    data=dict(size=1),
    pipeline=dict(size=1),
    tensor=dict(size=1, mode=None),
)))
colossalai.init_dist(config=parallel_cfg,
          local_rank=0,
          world_size=1,
          host='127.0.0.1',
          port=8888,
          backend='nccl')

colossalai - torch.distributed.distributed_c10d - 2021-12-04 10:10:36,754 INFO: Added key: store_based_barrier_key:1 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-04 10:10:36,756 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-04 10:10:36,764 INFO: Added key: store_based_barrier_key:2 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-04 10:10:36,769 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 1 nodes.
colossalai - torch.distributed.distributed_c10d - 2021-12-04 10:10:36,773 INFO: Added key: store_based_barrier_key:3 to store for rank: 0
colossalai - torch.distributed.distributed_c10d - 2021-12-04 10:10:36,775 INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:3 with 1 nodes.


process rank 0 is bound to device 0


Download the Hyperspectral image dataset named IndianPines with 17 classes and its groundtruth in 'http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat' and 'http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat', respectively. Put them in your root directory of Google Cloud Drive and load them.

---



In [5]:
import numpy as np
import torch
import torch.utils
import os
from tqdm import tqdm
from sklearn import preprocessing
from scipy import io, misc

# Load the image
folder_Pine = './'
img = io.loadmat(folder_Pine + 'Indian_pines_corrected.mat')
img = img['indian_pines_corrected']
gt = io.loadmat(folder_Pine + 'Indian_pines_gt.mat')['indian_pines_gt']
LABEL_VALUES = ["Undefined", "Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]
IGNORED_LABELS = [0]
nan_mask = np.isnan(img.sum(axis=-1))
if np.count_nonzero(nan_mask) > 0:
  print("Warning: NaN have been found in the data. It is preferable to remove them beforehand. Learning on NaN data is disabled.")
img[nan_mask] = 0
gt[nan_mask] = 0
IGNORED_LABELS.append(0)
IGNORED_LABELS = list(set(IGNORED_LABELS))
# Normalization
img = np.asarray(img, dtype='float32')
#img = (img - np.min(img)) / (np.max(img) - np.min(img))
data = img.reshape(np.prod(img.shape[:2]), np.prod(img.shape[2:]))
#data = preprocessing.scale(data)
data  = preprocessing.minmax_scale(data)
img = data.reshape(img.shape)

# N_CLASSES = len(LABEL_VALUES) -  len(IGNORED_LABELS)
N_CLASSES = len(LABEL_VALUES)
# Number of bands (last dimension of the image tensor)
N_BANDS = img.shape[-1]

Define the generic class named HyperX for a hyperspectral scene.

In [6]:
class HyperX(torch.utils.data.Dataset):
    def __init__(self, data, gt, **hyperparams):
        super(HyperX, self).__init__()
        self.data = data
        self.label = gt
        self.name = hyperparams['dataset']
        self.patch_size = hyperparams['patch_size']
        self.ignored_labels = set(hyperparams['ignored_labels'])
        self.flip_augmentation = hyperparams['flip_augmentation']
        self.radiation_augmentation = hyperparams['radiation_augmentation'] 
        self.mixture_augmentation = hyperparams['mixture_augmentation'] 
        self.center_pixel = hyperparams['center_pixel']
        supervision = hyperparams['supervision']
        # Fully supervised : use all pixels with label not ignored
        if supervision == 'full':
            mask = np.ones_like(gt)
            for l in self.ignored_labels:
                mask[gt == l] = 0
        # Semi-supervised : use all pixels, except padding
        elif supervision == 'semi':
            mask = np.ones_like(gt)
        x_pos, y_pos = np.nonzero(mask)
        p = self.patch_size // 2
        self.indices = np.array([(x,y) for x,y in zip(x_pos, y_pos) if x > p and x < data.shape[0] - p and y > p and y < data.shape[1] - p])
        self.labels = [self.label[x,y] for x,y in self.indices]
        np.random.shuffle(self.indices)
    @staticmethod
    def flip(*arrays):
        horizontal = np.random.random() > 0.5
        vertical = np.random.random() > 0.5
        if horizontal:
            arrays = [np.fliplr(arr) for arr in arrays]
        if vertical:
            arrays = [np.flipud(arr) for arr in arrays]
        return arrays
    @staticmethod
    def radiation_noise(data, alpha_range=(0.9, 1.1), beta=1/25):
        alpha = np.random.uniform(*alpha_range)
        noise = np.random.normal(loc=0., scale=1.0, size=data.shape)
        return alpha * data + beta * noise
    def mixture_noise(self, data, label, beta=1/25):
        alpha1, alpha2 = np.random.uniform(0.01, 1., size=2)
        noise = np.random.normal(loc=0., scale=1.0, size=data.shape)
        data2 = np.zeros_like(data)
        for  idx, value in np.ndenumerate(label):
            if value not in self.ignored_labels:
                l_indices = np.nonzero(self.labels == value)[0]
                l_indice = np.random.choice(l_indices)
                assert(self.labels[l_indice] == value)
                x, y = self.indices[l_indice]
                data2[idx] = self.data[x,y]
        return (alpha1 * data + alpha2 * data2) / (alpha1 + alpha2) + beta * noise
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, i):
        x, y = self.indices[i]
        x1, y1 = x - self.patch_size // 2, y - self.patch_size // 2
        x2, y2 = x1 + self.patch_size, y1 + self.patch_size
        data = self.data[x1:x2, y1:y2]
        label = self.label[x1:x2, y1:y2]
        if self.flip_augmentation and self.patch_size > 1:
            # Perform data augmentation (only on 2D patches)
            data, label = self.flip(data, label)
        if self.radiation_augmentation and np.random.random() < 0.1:
                data = self.radiation_noise(data)
        if self.mixture_augmentation and np.random.random() < 0.2:
                data = self.mixture_noise(data, label)
        # Copy the data into numpy arrays (PyTorch doesn't like numpy views)
        data = np.asarray(np.copy(data), dtype='float32')
        label = np.asarray(np.copy(label), dtype='int64')
        data = np.expand_dims(data, axis=0)
        # Load the data into PyTorch tensors
        data = torch.from_numpy(data)
        label = torch.from_numpy(label)
        targets = label.view(1)

        return data, targets



Define the train_loader and test_loader for traning.

In [8]:
import sklearn.model_selection
import torch.utils.data as data
import numpy as np
from scipy import io
hyperparams = {'batch_size': 1, 
        'dataset': 'IndianPines',
        'patch_size': 1,
        'ignored_labels': [0],
        'flip_augmentation': False,
        'radiation_augmentation': False, 
        'mixture_augmentation': False,
        'center_pixel': True,
        'supervision': 'full'
        }

def sample_gt(gt, train_size, mode='random'):

    indices = np.nonzero(gt)
    X = list(zip(*indices)) 
    y1 = gt[indices]
    y = y1.ravel() 
    train_gt = np.zeros_like(gt) 
    test_gt = np.zeros_like(gt)
    if train_size > 1:
       train_size = int(train_size)
    if mode == 'random':
       train_indices, test_indices = sklearn.model_selection.train_test_split(X, train_size=train_size, stratify=y)
       train_indices = [list(t) for t in zip(*train_indices)]
       test_indices = [list(t) for t in zip(*test_indices)]
       train_gt[train_indices] = gt[train_indices]
       test_gt[test_indices] = gt[test_indices]
    return train_gt, test_gt
train_gt, test_gt = sample_gt(gt, 0.1, mode='random')
# Generate the dataset
train_dataset = HyperX(img, train_gt, **hyperparams)
test_dataset = HyperX(img, test_gt, **hyperparams)    
print("HSI train dataset")
print(train_dataset)
train_loader = data.DataLoader(train_dataset,
                batch_size=hyperparams['batch_size'],
                shuffle=True)
test_loader = data.DataLoader(test_dataset,
               batch_size=hyperparams['batch_size'])

HSI train dataset


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Define a simple NN-based network here.

In [9]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

class HSI_processing(nn.Module):
    """
    Simple NN-based network
    """
    @staticmethod
    def weight_init(m):
        if isinstance(m, nn.Linear):
            init.kaiming_normal_(m.weight)
            init.zeros_(m.bias)

    def __init__(self, input_channels=200, n_classes=17, dropout=False):
        super(HSI_processing, self).__init__()
        self.use_dropout = dropout
        if dropout:
            self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(input_channels, 2048)
        self.fc2 = nn.Linear(2048, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, n_classes)
        self.apply(self.weight_init)

    def forward(self, x):
        x = x.view(x.shape[0], -1) #(200,1)
        x = F.relu(self.fc1(x))   #(1,2048)
        if self.use_dropout:
            x = self.dropout(x)
        x = F.relu(self.fc2(x))
        if self.use_dropout:
            x = self.dropout(x)
        x = F.relu(self.fc3(x))
        if self.use_dropout:
            x = self.dropout(x)
        x = self.fc4(x)
        return x  #(1,17)

model = HSI_processing().cuda()

Define a Loss function and optimizer to initialize `Engine` and `Trainer`. Use the hook to compute and print loss and accuracy.

In [10]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
schedule = NoPipelineSchedule()

engine = Engine(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=None,
        schedule=schedule
    )
trainer = Trainer(engine=engine,
          hooks_cfg=[dict(type='LossHook'), dict(type='LogMetricByEpochHook'), dict(type='AccuracyHook')],
          verbose=True)

colossalai - rank_0 - 2021-12-04 10:11:48,617 WARNING: No gradient handler is set up, please make sure you do not need to all-reduce the gradients after a training step.
colossalai - rank_0 - 2021-12-04 10:11:48,626 INFO: build LogMetricByEpochHook for train, priority = 1
colossalai - rank_0 - 2021-12-04 10:11:48,628 INFO: build LossHook for train, priority = 10
colossalai - rank_0 - 2021-12-04 10:11:48,630 INFO: build AccuracyHook for train, priority = 10


Train model for 10 epochs and it will be evaluated every 3 epoch. 

In [11]:
num_epochs = 10
test_interval = 1
trainer.fit(
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        max_epochs=num_epochs,
        display_progress=True,
        test_interval=test_interval
    )

[Epoch 0 train]: 100%|██████████| 1020/1020 [00:13<00:00, 77.13it/s]
colossalai - rank_0 - 2021-12-04 10:12:04,413 INFO: Training - Epoch 1 - LogMetricByEpochHook: Loss = 1.81093
[Epoch 0 val]: 100%|██████████| 9156/9156 [00:12<00:00, 747.87it/s]
colossalai - rank_0 - 2021-12-04 10:12:16,674 INFO: Testing - Epoch 1 - LogMetricByEpochHook: Loss = 1.42704, Accuracy = 0.48012
[Epoch 1 train]: 100%|██████████| 1020/1020 [00:13<00:00, 76.09it/s]
colossalai - rank_0 - 2021-12-04 10:12:30,103 INFO: Training - Epoch 2 - LogMetricByEpochHook: Loss = 1.43416
[Epoch 1 val]: 100%|██████████| 9156/9156 [00:11<00:00, 801.98it/s]
colossalai - rank_0 - 2021-12-04 10:12:41,528 INFO: Testing - Epoch 2 - LogMetricByEpochHook: Loss = 1.40988, Accuracy = 0.49792
[Epoch 2 train]: 100%|██████████| 1020/1020 [00:13<00:00, 76.19it/s]
colossalai - rank_0 - 2021-12-04 10:12:54,933 INFO: Training - Epoch 3 - LogMetricByEpochHook: Loss = 1.31126
[Epoch 2 val]: 100%|██████████| 9156/9156 [00:12<00:00, 757.60it/s]
c